In [2]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import nsdiffs, ndiffs
from pmdarima import auto_arima

In [ ]:
file_path = r"C:\Users\huawei\OneDrive\桌面\original.xlsx"

# 读取数据，time列解析为日期但不设为索引
df = pd.read_excel(file_path, parse_dates=['time'])

# 按时间列从小到大排序
df = df.sort_values('time').reset_index(drop=True)

print(df.head(3))
print(df.dtypes)

        time  GDP_4cum  city_mean_light  region_mean_light title website  \
0 2008-01-01   1634.65        13.180645            0.98249  (填补)    none   
1 2008-04-01   3346.63        13.180645            0.98249  (填补)    none   
2 2008-07-01   5351.74        13.180645            0.98249  (填补)    none   

   Year  Month      GDP    ln_gdp   ln_city  ln_region  
0  2008      1  1634.65  7.399184  2.578749  -0.017665  
1  2008      4  1711.98  7.445406  2.578749  -0.017665  
2  2008      7  2005.11  7.603454  2.578749  -0.017665  
time                 datetime64[ns]
GDP_4cum                    float64
city_mean_light             float64
region_mean_light           float64
title                        object
website                      object
Year                          int64
Month                         int64
GDP                         float64
ln_gdp                      float64
ln_city                     float64
ln_region                   float64
dtype: object


In [58]:
# 筛选year列为2014到2023的数据，并创建副本
cal = df[(df['Year'] >= 2014) & (df['Year'] <= 2023)].copy()

print(f"筛选的年份范围: {cal['Year'].min()} - {cal['Year'].max()}")
print(cal.head(2))

筛选的年份范围: 2014 - 2023
         time   GDP_4cum  city_mean_light  region_mean_light        title  \
24 2014-01-01  2953.2500        14.497979           1.437986  一季度深圳经济运行情况   
25 2014-04-01  6494.8716        14.614505           1.380323  上半年深圳经济运行情况   

                                              website  Year  Month        GDP  \
24  https://tjj.sz.gov.cn/zwgk/zfxxgkml/tjsj/tjfx/...  2014      1  2953.2500   
25  https://tjj.sz.gov.cn/zwgk/zfxxgkml/tjsj/tjfx/...  2014      4  3541.6216   

      ln_gdp   ln_city  ln_region  
24  7.990662  2.674009   0.363243  
25  8.172340  2.682015   0.322318  


In [59]:
# 按年份分组，计算每个季度灯光值占全年的比例
cal['city_prop'] = cal.groupby('Year')['city_mean_light'].transform(
    lambda x: x / x.sum()
)

# 验证结果
print("每年各季度city_prop的总和（应该都等于1）:")
print(cal.groupby('Year')['city_prop'].sum())

print("\n前几行数据:")
print(cal[['Year', 'city_mean_light', 'city_prop']].head())

每年各季度city_prop的总和（应该都等于1）:
Year
2014    1.0
2015    1.0
2016    1.0
2017    1.0
2018    1.0
2019    1.0
2020    1.0
2021    1.0
2022    1.0
2023    1.0
Name: city_prop, dtype: float64

前几行数据:
    Year  city_mean_light  city_prop
24  2014        14.497979   0.241034
25  2014        14.614505   0.242971
26  2014        15.858605   0.263655
27  2014        15.178015   0.252340
28  2015        14.593800   0.240887


In [60]:
# 按照Month列分组（4个季度），计算city_prop的平均值
quarterly_city_prop = cal.groupby('Month')['city_prop'].mean().round(4)

print(quarterly_city_prop)

# 添加季度标签显示
quarterly_labels = {
    1: 'Q1 (1月)',
    4: 'Q2 (4月)', 
    7: 'Q3 (7月)',
    10: 'Q4 (10月)'
}

print("\n带标签的各季度city_prop平均值:")
for month, avg_prop in quarterly_city_prop.items():
    quarter_label = quarterly_labels[month]
    print(f"{quarter_label}: {avg_prop:.4f}")

# 验证：所有季度平均值之和应该等于1
total_avg = quarterly_city_prop.sum()
print(f"\n验证：所有季度平均值之和 = {total_avg:.4f}")

Month
1     0.2381
4     0.2519
7     0.2502
10    0.2598
Name: city_prop, dtype: float64

带标签的各季度city_prop平均值:
Q1 (1月): 0.2381
Q2 (4月): 0.2519
Q3 (7月): 0.2502
Q4 (10月): 0.2598

验证：所有季度平均值之和 = 1.0000


In [61]:
# 按年份分组，计算每个季度灯光值占全年的比例
cal['region_prop'] = cal.groupby('Year')['region_mean_light'].transform(
    lambda x: x / x.sum()
)

# 验证结果
print("每年各季度city_prop的总和（应该都等于1）:")
print(cal.groupby('Year')['region_prop'].sum())

print("\n前几行数据:")
print(cal[['Year', 'region_mean_light', 'region_prop']].head())

每年各季度city_prop的总和（应该都等于1）:
Year
2014    1.0
2015    1.0
2016    1.0
2017    1.0
2018    1.0
2019    1.0
2020    1.0
2021    1.0
2022    1.0
2023    1.0
Name: region_prop, dtype: float64

前几行数据:
    Year  region_mean_light  region_prop
24  2014           1.437986     0.245767
25  2014           1.380323     0.235912
26  2014           1.558785     0.266413
27  2014           1.473916     0.251908
28  2015           1.491912     0.249547


In [62]:
# 按照Month列分组（4个季度），计算region_prop的平均值
quarterly_region_prop = cal.groupby('Month')['region_prop'].mean().round(4)

print(quarterly_region_prop)

# 添加季度标签显示
quarterly_labels = {
    1: 'Q1 (1月)',
    4: 'Q2 (4月)', 
    7: 'Q3 (7月)',
    10: 'Q4 (10月)'
}
print("\n带标签的各季度region_prop平均值:")
for month, avg_prop in quarterly_region_prop.items():
    quarter_label = quarterly_labels[month]
    print(f"{quarter_label}: {avg_prop:.4f}")

# 验证：所有季度平均值之和应该等于1
total_avg = quarterly_region_prop.sum()
print(f"\n验证：所有季度平均值之和 = {total_avg:.4f}")

Month
1     0.2425
4     0.2511
7     0.2475
10    0.2589
Name: region_prop, dtype: float64

带标签的各季度region_prop平均值:
Q1 (1月): 0.2425
Q2 (4月): 0.2511
Q3 (7月): 0.2475
Q4 (10月): 0.2589

验证：所有季度平均值之和 = 1.0000


In [63]:
# 筛选2008-2013年的数据
data_2008_2013 = df[(df['Year'] >= 2008) & (df['Year'] <= 2013)].copy()

# 将两部分数据合并
cal = pd.concat([cal, data_2008_2013], ignore_index=True)

# 按时间排序
cal = cal.sort_values('time').reset_index(drop=True)

In [65]:
# 为2008-2013年的数据填补city_prop
cal.loc[cal['Year'].between(2008, 2013), 'city_prop'] = cal.loc[cal['Year'].between(2008, 2013), 'Month'].map(quarterly_city_prop)

# 为2008-2013年的数据填补region_prop
cal.loc[cal['Year'].between(2008, 2013), 'region_prop'] = cal.loc[cal['Year'].between(2008, 2013), 'Month'].map(quarterly_region_prop)


# 检查填补后的结果
print(cal[cal['Year'].between(2008, 2013)][['Year', 'Month', 'city_prop', 'region_prop']].head())

# 验证每年的比例总和是否为1
print("\n验证各年份比例总和:")
print(cal[cal['Year'].between(2008, 2013)].groupby('Year')[['city_prop', 'region_prop']].sum())

   Year  Month  city_prop  region_prop
0  2008      1     0.2381       0.2425
1  2008      4     0.2519       0.2511
2  2008      7     0.2502       0.2475
3  2008     10     0.2598       0.2589
4  2009      1     0.2381       0.2425

验证各年份比例总和:
      city_prop  region_prop
Year                        
2008        1.0          1.0
2009        1.0          1.0
2010        1.0          1.0
2011        1.0          1.0
2012        1.0          1.0
2013        1.0          1.0


In [66]:
# 计算2008-2013年每年的年度总和
yearly_sums = cal[cal['Year'].between(2008, 2013)].groupby('Year').agg({
    'city_mean_light': 'sum',
    'region_mean_light': 'sum'
}).reset_index()

# 重命名列名
yearly_sums.rename(columns={
    'city_mean_light': 'city_yearly_sum',
    'region_mean_light': 'region_yearly_sum'
}, inplace=True)

print("各年度总和:")
print(yearly_sums)

各年度总和:
   Year  city_yearly_sum  region_yearly_sum
0  2008        52.722581           3.929959
1  2009        54.207924           4.045277
2  2010        58.223983           5.320330
3  2011        60.451682           5.554874
4  2012        57.671480           5.319744
5  2013        59.795041           5.931304


In [67]:
# 将年度总和合并到cal数据中
cal = cal.merge(yearly_sums, on='Year', how='left')

# 检查合并结果
print("\n合并后的数据示例:")
print(cal[cal['Year'].between(2008, 2013)][['Year', 'Month', 'city_yearly_sum', 'region_yearly_sum']].head())


合并后的数据示例:
   Year  Month  city_yearly_sum  region_yearly_sum
0  2008      1        52.722581           3.929959
1  2008      4        52.722581           3.929959
2  2008      7        52.722581           3.929959
3  2008     10        52.722581           3.929959
4  2009      1        54.207924           4.045277


In [71]:
# 只对2008-2013年的数据进行重新分配
mask_2008_2013 = cal['Year'].between(2008, 2013)

# 用city_prop比例重新分配city_mean_light
cal.loc[mask_2008_2013, 'city_mean_light'] = cal.loc[mask_2008_2013, 'city_yearly_sum'] * cal.loc[mask_2008_2013, 'city_prop']

# 用region_prop比例重新分配region_mean_light
cal.loc[mask_2008_2013, 'region_mean_light'] = cal.loc[mask_2008_2013, 'region_yearly_sum'] * cal.loc[mask_2008_2013, 'region_prop']


# 验证重新分配后的结果
print("重新分配后的数据:")
print(cal[cal['Year'].between(2008, 2013)][['Year', 'Month', 'city_mean_light', 'region_mean_light']].head(8))

重新分配后的数据:
   Year  Month  city_mean_light  region_mean_light
0  2008      1        12.553247           0.953015
1  2008      4        13.280818           0.986813
2  2008      7        13.191190           0.972665
3  2008     10        13.697327           1.017466
4  2009      1        12.906907           0.980980
5  2009      4        13.654976           1.015769
6  2009      7        13.562823           1.001206
7  2009     10        14.083219           1.047322


In [ ]:
# 将数据保存为CSV文件
output_path = r"C:\Users\huawei\OneDrive\桌面\limited.csv"
cal.to_csv(output_path, index=False, encoding='utf-8-sig')